In [507]:
import torch
from PIL import Image
import clip
import os.path as osp
import os, sys
sys.path.insert(0, '../')
sys.path.append('.')
sys.path.append('..')
import shutil

import torchvision.utils as vutils
from lib.utils import load_model_weights,mkdir_p
from models.GALIP import NetG, CLIP_TXT_ENCODER
import argparse
import cv2
import glob
import numpy as np
import torch
from basicsr.utils import imwrite
from gfpgan import GFPGANer



In [508]:
device = 'cpu' # 'cpu' # 'cuda:0'
CLIP_text = "ViT-B/32"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model = clip_model.eval()

In [509]:
text_encoder = CLIP_TXT_ENCODER(clip_model).to(device)
netG = NetG(64, 100, 512, 256, 3, False, clip_model).to(device)

In [510]:
text_encoder = CLIP_TXT_ENCODER(clip_model).to(device)
netG = NetG(64, 100, 512, 256, 3, False, clip_model).to(device)
path = '../saved_models/state_epoch_1160.pth'
# path = '/home/luxj/document/GALIP-main/code/saved_models/faces/GALIP_nf64_normal_faces_256_2024_03_03_17_39_53/state_epoch_620.pth'
checkpoint = torch.load(path, map_location=torch.device('cpu'))
netG = load_model_weights(netG, checkpoint['model']['netG'], multi_gpus=False)

In [511]:
# 清空文件夹
def clear_folder(folder_path):
    if not os.path.exists(folder_path):
        return

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        if os.path.isfile(file_path):
            os.remove(file_path)
        # 清空
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

In [512]:
batch_size = 4 #需要生成多少张图片
noise = torch.randn((batch_size, 100)).to(device)

In [513]:
# 输入语料
captions = ['She is young, and slim with quite small lips, high cheekbones, bags under eyes, straight eyebrows, no bangs. She has brown hair. She has no beard, a normal chin.  She doesn\'t put neither lipstick, earrings, necklace, eyeglasses, hat, heavy makeup nor neck tie. ']

In [514]:
mkdir_p('./samples') 
folder_path = './samples' # 清空
clear_folder(folder_path)                                     

In [515]:
# # generate from text
# with torch.no_grad():
#     for i in range(len(captions)):
#         caption = captions[i]
#         tokenized_text = clip.tokenize([caption]).to(device)
#         sent_emb, word_emb = text_encoder(tokenized_text)
#         sent_emb = sent_emb.repeat(batch_size,1)
#         fake_imgs = netG(noise,sent_emb,eval=True).float()
#         # name = f'{captions[i].replace(" ", "-")}'
#         name = 'demo4'
#         vutils.save_image(fake_imgs.data, './samples/%s.png'%(name), nrow=8, value_range=(-1, 1), normalize=True)

In [516]:
with torch.no_grad():
    for i in range(len(captions)):
        caption = captions[i]
        tokenized_text = clip.tokenize([caption]).to(device)
        sent_emb, word_emb = text_encoder(tokenized_text)
        sent_emb = sent_emb.repeat(batch_size, 1)
        fake_imgs = netG(noise, sent_emb, eval=True).float()
        # batch_size = 1 #需要生成多少张图片
        # 分开保存
        for j in range(batch_size):
            fake_img = fake_imgs[j]
            name = 'demo_%d' % (i * batch_size + j + 1)
            vutils.save_image(fake_img, './samples/%s.png' % (name), value_range=(-1, 1), normalize=True)

python inference_gfpgan.py -i inputs/faces -o results -v 1.3 -s 2

In [517]:
# args setting
input = "./samples"
mkdir_p('./results')
output = './results'
bg_upsampler = 'realesrgan'
tile = 400
version = 1.3 #默认版本1.3 ，共有1 | 1.2 | 1.3
upscale = 2 
weight = 0.5
suffix = None
ext = 'auto'

In [518]:
# ------------------------ input & output ------------------------
if input.endswith('/'):
    input = input[:-1]
if os.path.isfile(input):
    img_list = [input]
else:
    img_list = sorted(glob.glob(os.path.join(input, '*')))
# os.makedirs(output, exist_ok=True)

In [519]:
 # ------------------------ set up background upsampler ------------------------
if bg_upsampler == 'realesrgan':
    if not torch.cuda.is_available():  # CPU
        import warnings
        warnings.warn('The unoptimized RealESRGAN is slow on CPU. We do not use it. '
                        'If you really want to use it, please modify the corresponding codes.')
        bg_upsampler = None
    else:
        from basicsr.archs.rrdbnet_arch import RRDBNet
        from realesrgan import RealESRGANer
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
        bg_upsampler = RealESRGANer(
            scale=2,
            model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
            model=model,
            tile=tile, # 超参数默认400
            tile_pad=10,
            pre_pad=0,
            half=True)  # need to set False in CPU mode
else:
    bg_upsampler = None

In [520]:
# ------------------------ set up GFPGAN restorer ------------------------
arch = 'clean'
channel_multiplier = 2
# model_name = 'GFPGANv1.3'
model_path = './pt/GFPGANv1.3.pth'

restorer = GFPGANer(
    model_path=model_path,
    upscale= upscale,
    arch=arch,
    channel_multiplier=channel_multiplier,
    bg_upsampler=bg_upsampler)

In [521]:
# ------------------------ restore ------------------------

for img_path in img_list:
    # read image
    img_name = os.path.basename(img_path)
    print(f'Processing {img_name} ...')
    basename, ext = os.path.splitext(img_name)
    input_img = cv2.imread(img_path, cv2.IMREAD_COLOR)

    # restore faces and background if necessary
    cropped_faces, restored_faces, restored_img = restorer.enhance(
        input_img,
        has_aligned='store_true',
        only_center_face='store_true',
        paste_back=True,
        weight=weight)

    # save faces
    for idx, (cropped_face, restored_face) in enumerate(zip(cropped_faces, restored_faces)):
        # save cropped face
        # save_crop_path = os.path.join(output, 'cropped_faces', f'{basename}_{idx:02d}.png')
        # imwrite(cropped_face, save_crop_path)

        # save restored face
        if suffix is not None:
            save_face_name = f'{basename}_{idx:02d}_{suffix}.png'
        else:
            save_face_name = f'{basename}_{idx:02d}.png'
        save_restore_path = os.path.join(output, 'restored_faces', save_face_name)
        imwrite(restored_face, save_restore_path)

        # save comparison image
        cmp_img = np.concatenate((cropped_face, restored_face), axis=1)
        imwrite(cmp_img, os.path.join(output, 'cmp', f'{basename}_{idx:02d}.png'))

    # save restored img
    if restored_img is not None:
        if ext == 'auto':
            extension = ext[1:]
        else:
            extension = ext

        if suffix is not None:
            save_restore_path = os.path.join(output, 'restored_imgs', f'{basename}_{suffix}.{extension}')
        else:
            save_restore_path = os.path.join(output, 'restored_imgs', f'{basename}.{extension}')
        imwrite(restored_img, save_restore_path)

print(f'Results are in the [{output}] folder.')

Processing demo_1.png ...
Processing demo_2.png ...
Processing demo_3.png ...
Processing demo_4.png ...
Results are in the [./results] folder.
